In [22]:
# %load mytest.py
from model import DeepHomographyModel
import torch
import cv2
import random
import numpy as np
from numpy.linalg import inv

net = DeepHomographyModel()

net.load_state_dict(torch.load('model_final_Vanilla'))

img = cv2.imread('handwriting1.tif', 0)
img = cv2.resize(img, (320, 240))

test_image = img.copy()

top_x = random.randint(0+32, 320-32-128)   #The structure of this part of the code is cited from  https://github.com/paeccher/Deep-Homography-Estimation-Pytorch/blob/master/dataset.py
top_y = random.randint(0+32, 240-32-128)

top_left_point = (top_x, top_y)
top_right_point = (top_x+128, top_y)
bottom_left_point = (top_x, top_y+128)

bottom_right_point = (top_x+128, top_y+128)

four_points = [top_left_point, top_right_point, bottom_left_point, bottom_right_point]

perturbed_four_points = []
for point in four_points:
    perturbed_four_points.append( (point[0] + random.randint(-32,32), point[1] + random.randint(-32,32)) )

H = cv2.getPerspectiveTransform( np.float32(four_points), np.float32(perturbed_four_points) )

H_inverse = inv(H)

warped_image = cv2.warpPerspective(img, H_inverse, (320,240))   #The citation ends here, note that only the structure was cited, the parameters are self-designed

Ip1 = test_image[top_left_point[1]:bottom_right_point[1],top_left_point[0]:bottom_right_point[0]]
Ip2 = warped_image[top_left_point[1]:bottom_right_point[1],top_left_point[0]:bottom_right_point[0]]
training_image = np.dstack((Ip1, Ip2))
H_four_points = np.subtract(np.array(perturbed_four_points), np.array(four_points))

datum = (training_image, H_four_points)
tensortraining_image = torch.from_numpy(training_image)
tensortraining_image = tensortraining_image.unsqueeze(0)
tensortraining_image = tensortraining_image.permute(0,3,1,2).float()

outputs = net.forward(tensortraining_image)
outputs = outputs

In [54]:
cv2.imshow('Image',img) #This part shows the original sample image
cv2.waitKey()
cv2.destroyAllWindows()

In [55]:
cv2.imshow('WarpedImage',warped_image) #This part shows the warped image that is also used as the input of the network
cv2.waitKey()
cv2.destroyAllWindows()

In [25]:
padimg = cv2.copyMakeBorder( img, 75, 75, 75, 75,cv2.BORDER_CONSTANT)

In [47]:
cv2.imshow('padImage',padimg) #This part shows the padded original image
cv2.waitKey()
cv2.destroyAllWindows()

In [27]:
padshiyan=cv2.warpPerspective(padimg,H_inverse,(320+500,240+500))

In [1]:
cv2.imshow('Image',padshiyan) #This part shows the warped padimg, the transformation matrix is exactly the same as the one above
cv2.waitKey()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [29]:
outputs #This part shows the output of the network, the transformation matrix

tensor([[ 10.2902,  16.3866, -30.2348,   8.1603, -36.9843,  42.8587,  22.2921,
          13.1488]], grad_fn=<AddmmBackward>)

In [30]:
npoutputs=outputs.detach().numpy()

In [31]:
npoutputs

array([[ 10.290247,  16.38657 , -30.234777,   8.160304, -36.98431 ,
         42.85875 ,  22.292082,  13.148786]], dtype=float32)

In [32]:
a1=np.matrix([(0,0),(320-1,0),(0,240-1),(320-1,240-1)])
a2=np.matrix([(0+npoutputs[0,0],0+npoutputs[0,1]),(320-1+npoutputs[0,2],0+npoutputs[0,3]),(0+npoutputs[0,4],240-1+npoutputs[0,5]),(320-1+npoutputs[0,6],240-1+npoutputs[0,7])])

In [33]:
transformationmatrix=cv2.getPerspectiveTransform( np.float32(a1), np.float32(a2) )

In [34]:
transformationmatrix #This part shows the equivalent 3x3 tranformation matrix

array([[ 9.22745789e-01, -1.55792789e-01,  1.02902470e+01],
       [-2.43808214e-02,  7.90612467e-01,  1.63865700e+01],
       [ 1.72401174e-04, -1.13585209e-03,  1.00000000e+00]])

In [35]:
Itransformationmatrix=inv(transformationmatrix)

In [36]:
Itransformationmatrix #This part shows the inverse of the transformation matrix

array([[ 1.09155173e+00,  1.94380523e-01, -1.44175669e+01],
       [ 3.66976111e-02,  1.24228498e+00, -2.07344172e+01],
       [-1.46501742e-04,  1.37754055e-03,  9.78934374e-01]])

In [40]:
###############################################################

In [37]:
padshiyan1=cv2.warpPerspective(padimg,transformationmatrix,(320+500,240+500))

In [58]:
cv2.imshow('padshiyan1',padshiyan1)
cv2.waitKey()
cv2.destroyAllWindows()

In [39]:
padshiyan2=cv2.warpPerspective(padimg,Itransformationmatrix,(320+500,240+500))

In [59]:
cv2.imshow('Image',padshiyan2)#We try to map the warped image to the original image. This part shows the result.
cv2.waitKey()
cv2.destroyAllWindows()

In [20]:
restore1=cv2.warpPerspective(padshiyan,transformationmatrix,(320+500,240+500))

In [41]:
cv2.imshow('restore1',restore1)#We try to map the original image to the warped image. This part shows the result.
cv2.waitKey()
cv2.destroyAllWindows()